In [1]:
import PIL.Image
import dlib
import numpy as np
from PIL import ImageFile
import face_recognition_models
import cv2
import glob
import os

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
# Pretrained weights of models used in dlib

predictor_5_point_model = face_recognition_models.pose_predictor_five_point_model_location()
pose_predictor_5_point = dlib.shape_predictor(predictor_5_point_model)
'''
This object is a tool that takes in an image region containing some object and outputs a set of point 
locations that define the pose of the object. The classic example of this is human face pose prediction,
where you take an image of a human face as input and are expected to identify the locations of important 
facial landmarks such as the corners of the mouth and eyes, tip of the nose, and so forth.
'''

face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)

'''
This object maps human faces into 128D vectors where pictures of the same person are mapped near to 
each other and pictures of different people are mapped far apart. The constructor loads the face 
recognition model from a file
'''

'\nThis object maps human faces into 128D vectors where pictures of the same person are mapped near to \neach other and pictures of different people are mapped far apart. The constructor loads the face \nrecognition model from a file\n'

In [4]:
def load_image_file(file):
    im = PIL.Image.open(file)
    im = im.convert('RGB')
    return np.array(im)

In [5]:
# This object represents a rectangular area of an image.

def css_to_rect(css):
    return dlib.rectangle(css[3], css[0], css[1], css[2])

In [6]:
# Get default face detector object to locate the face locations

def raw_face_location(img, number_of_times_to_upsample=1, model="hog"):
    
    face_detector = dlib.get_frontal_face_detector()
    return face_detector(img, number_of_times_to_upsample)

In [7]:
# This function returns the 5 point face location estimate using dlib libraries

def raw_face_landmarks(face_image, face_locations=None, model="large"):
    if face_locations is None:
        face_locations = raw_face_location(face_image)
    else:
        face_locations = [css_to_rect(face_location) for face_location in face_locations]

    pose_predictor = pose_predictor_5_point


    return [pose_predictor(face_image, face_location) for face_location in face_locations]

In [8]:
# encodes face into 128D vector

def encode_face(face_image, known_face_locations=None, num_jitters=1, model="small"):
    
    raw_landmarks = raw_face_landmarks(face_image, known_face_locations, model)
    return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)) for raw_landmark_set in raw_landmarks]

In [9]:
# Takes each image from the folder and encodes each of them as known face encodings
known_face_encodings = []
known_face_names = []
for image in glob.glob('./images/*'):
    img = load_image_file(image)
    enc = encode_face(img)[0]
    known_face_encodings.append(enc)
    known_face_names.append(os.path.splitext(os.path.basename(image))[0])

In [10]:
known_face_names

['Elon', 'Obama', 'Sanjay', 'TIm']

In [11]:
# convert dlib rect format to css
def rect_to_css(rect):
    return rect.top(), rect.right(), rect.bottom(), rect.left()

#Trim excess bounds to get the face location
def trim_css_to_bounds(css, image_shape):

    return max(css[0], 0), min(css[1], image_shape[1]), min(css[2], image_shape[0]), max(css[3], 0)

#inputs and data and returns the face locations by calling raw face location function
def get_face_locations(img, number_of_times_to_upsample=1, model="hog"):

    return [trim_css_to_bounds(rect_to_css(face), img.shape) for face in raw_face_location(img, number_of_times_to_upsample, model)]

In [12]:
# Calculate the vector difference between the known 128D vector and the unknown vector

def face_distance(face_encodings, face_to_compare):
    if len(face_encodings) == 0:
        return np.empty((0))

    return np.linalg.norm(face_encodings - face_to_compare, axis=1)

# list out all the face encoding for a certain threshold

def compare_faces(known_face_encodings, face_encoding_to_check, tolerance=0.6):
    return list(face_distance(known_face_encodings, face_encoding_to_check) <= tolerance)

In [13]:
# import numpy as np
# import cv2

# cap = cv2.VideoCapture(0)

# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Display the resulting frame
#     cv2.imshow('frame',gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

In [19]:
video_capture = cv2.VideoCapture(0)
while(True):
    # Grab a single frame of video
    ret, frame = video_capture.read()
    frameorg= frame
    
    
    # Converting from BGR to RGB
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = get_face_locations(rgb_frame)
    face_encodings = encode_face(rgb_frame, face_locations)
    
    #draw face locations
    for loc in face_locations:
        cv2.rectangle(frameorg, (loc[1], loc[0]), (loc[3], loc[2]), (0, 0, 255), 2)
    cv2.imshow('original fram',frameorg)
    
    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # human face with the smallest distance but which passes a default threshold is selected
        face_distances = face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frameorg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows() 

In [15]:
! pip install matplotlib


In [16]:
fig=plt.figure(figsize=(8, 8))
rows = min(4,len(face_encodings))
columns = len(face_encodings)//4 +1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(face_encodings[i].reshape(16,8), cmap='gray')

NameError: name 'plt' is not defined

In [ ]:
columns
